In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import numpy as np
from pathlib import Path
import plotly
import plotly.graph_objs as go
import plotly.express as px
plotly.offline.init_notebook_mode() # inline
import canonicalize_camera_poses

In [ ]:
root = Path('/home/gerry/Dropbox/Hydroponics/hyperspectral/GTRI_proof_of_concept/results_gerry/nerfstudio/test')
data = json.load(open(root / 'transforms.json', 'r'))

## First plot initial example data

In [ ]:
frames = data['frames']
frames = sorted(data['frames'], key=lambda frame: frame['file_path'])
Ts = np.array([frame['transform_matrix'] for frame in frames])

In [ ]:
fig = px.scatter_3d(x=Ts[:, 0, 3], y=Ts[:, 1, 3], z=Ts[:, 2, 3], width=500, height=300)
fig.update_layout(margin=dict(l=20, r=20, t=20, b=20))
fig.show()

## Try to compute / correct for rotation, translation, and scale

In [ ]:
# Vector between first image and the farthest away image should be x-axis.
ts = Ts[:, :3, 3]
dists = np.sqrt(np.sum(np.square(ts[:,:,None] - ts.T[None, :, :]), axis=1))
farthest_image = np.argmax(dists[0][:25])
print(farthest_image)
# JK I don't want to take this approach

In [ ]:
# Find (and remove) duplicate images
ts = Ts[:, :3, 3]
dists = np.sqrt(np.sum(np.square(ts[:,:,None] - ts.T[None, :, :]), axis=1))
dists /= np.max(dists)
# max distance is radius.  Image every 15 degrees means nearest distance should be d*pi*(15°/360°)
np.fill_diagonal(dists, 1)  # Ignore diagonal
px.imshow(dists < np.pi * (15/360) / 2, height=300).show()

In [ ]:
duplicates = np.argwhere(dists < np.pi * (15/360) / 2)
print(duplicates)
to_take = list(range(len(frames)))
for _, dup in filter(lambda dup: dup[0] < dup[1], duplicates):
    to_take.remove(dup)
print(len(to_take), to_take)
assert len(to_take) == 48, 'An unexpected number of camera poses was found'

# Find which is the bigger circle
if dists[to_take[0], to_take[11]] > dists[to_take[24], to_take[35]]:
    big_circle = to_take[:24]
else:
    big_circle = to_take[24:]

theta = -(np.arange(0, 360, 15) * np.pi / 180).reshape(-1, 1)
expected = np.hstack((np.cos(theta), np.sin(theta), np.zeros_like(theta)))
_, sR, t = canonicalize_camera_poses.ICP_transform_with_scale(ts[big_circle], expected)
print(sR, t)

In [ ]:
T = np.zeros((4, 4))
T[:3, :3] = sR
T[:3, 3] = t
T[3, 3] = 1
print(T)
print(T.shape, Ts.shape)
new_Ts = T @ Ts
print(new_Ts.shape)
fig = px.scatter_3d(x=new_Ts[:, 0, 3], y=new_Ts[:, 1, 3], z=new_Ts[:, 2, 3], width=500, height=300)
fig.update_layout(scene=dict(aspectmode='data'))
fig.show()

### Implement in `canonicalize_camera_poses` and test

In [ ]:
frames = canonicalize_camera_poses.canonicalize_camera_poses(data['frames'])
Ts = np.array([frame['transform_matrix'] for frame in frames])
fig = px.scatter_3d(x=Ts[:, 0, 3], y=Ts[:, 1, 3], z=Ts[:, 2, 3], width=500, height=300)
fig.update_layout(margin=dict(l=20, r=20, t=20, b=20), scene=dict(aspectmode='data'))
fig.show()

## Finally, correct so that the subject is in the center
(The cameras are looking slightly downwards)

In [ ]:
Ts = np.array([frame['transform_matrix'] for frame in frames])
fx, fy, cx, cy = data['fl_x'], data['fl_y'], data['cx'], data['cy']

R, t = Ts[0, :3, :3], Ts[0, :3, 3]
ray_angle = R[:, 2].reshape(-1, 1)
ray_angle /= np.linalg.norm(ray_angle)
s = np.linspace(0, 1)
ray = (t[:, None] + -ray_angle * s).T
# px.line_3d(x=ray[:, 0], y = ray[:, 1], z=ray[:, 2]).show()
fig.add_scatter3d(x=ray[:, 0], y = ray[:, 1], z=ray[:, 2], mode='lines')
fig.show()

In [ ]:
# Denote v := closest vector along the ray that ends at the z-axis.
# v_xy = proj(x_xy -> ray_xy)
# v_xy = proj(v -> xy)
x_xy = -t[:2]
ray_angle = ray_angle.squeeze()
ray_xy = ray_angle[:2]
v_xy = np.dot(x_xy, ray_xy) * ray_xy / np.sum(np.square(ray_xy))
print(x_xy, ray_xy, v_xy)
assert v_xy[0] / ray_angle[0] == v_xy[1] / ray_angle[1], 'v_xy is not a projection of v onto xy'
dz_height = ray_angle[2] * v_xy[0] / ray_angle[0]
z_height = dz_height + t[2]
v_xy = np.array([*v_xy, dz_height])
print(z_height, dz_height, v_xy)
# closest_vec_to_z_axis = 
# z_height = 
end_of_ray = t + v_xy
fig.add_scatter3d(x=[end_of_ray[0]], y = [end_of_ray[1]], z=[end_of_ray[2]])

Vectorized version

In [ ]:
ray_dirs, ts = Ts[:, :3, 2], Ts[:, :3, 3]

x_xy = -ts[:, :2]
ray_xy = ray_dirs[:, :2]
v_xy = np.sum(x_xy * ray_xy, axis=1)[:, None] * ray_xy / np.sum(np.square(ray_xy), axis=1)[:, None]
ratios = v_xy / ray_xy
np.testing.assert_allclose(*ratios.T, err_msg='v_xy is not a projection of v onto xy')
dz_height = ray_dirs[:, 2] * ratios[:, 0]
v_xy = np.hstack((v_xy, dz_height[:, None]))
z_height = dz_height + ts[:, 2]
print(z_height)
z_height = np.mean(z_height)
print(z_height)

ts_new = ts
ts_new[:, 2] -= z_height

In [ ]:
# Plot
end_of_ray = ts + v_xy
rays = np.stack((ts, ts - ray_dirs / np.linalg.norm(ray_dirs, axis=1, keepdims=True)), axis=2)

fig = px.scatter_3d(x=ts[:, 0], y=ts[:, 1], z=ts[:, 2], width=500, height=300)
for ray in rays:
    fig.add_scatter3d(x=ray[0, :], y = ray[1, :], z=ray[2, :], mode='lines', line=dict(color='red', width=1))
fig.add_scatter3d(x=end_of_ray[:, 0], y = end_of_ray[:, 1], z=end_of_ray[:, 2], marker=dict(color='green', size=4))
fig.update_layout(margin=dict(l=20, r=20, t=20, b=20), scene=dict(aspectmode='data'))
fig.show()

### And rescale so that the FOV is unit block centered at origin
(-0.5, 0.5)

In [ ]:
radius = 1
max_x, max_y = radius * cx / fx, radius * cy / fy
print(max_x, max_y)
scale_factor = 0.5 / max(max_x, max_y)
ts_new *= scale_factor

In [ ]:
# Plot
end_of_ray = ts_new + v_xy * scale_factor
rays = np.stack((ts_new, ts_new - ray_dirs / np.linalg.norm(ray_dirs, axis=1, keepdims=True) * scale_factor), axis=2)

fig = px.scatter_3d(x=ts_new[:, 0], y=ts_new[:, 1], z=ts_new[:, 2], width=500, height=300)
for ray in rays:
    fig.add_scatter3d(x=ray[0, :], y = ray[1, :], z=ray[2, :], mode='lines', line=dict(color='red', width=1))
fig.add_scatter3d(x=end_of_ray[:, 0], y = end_of_ray[:, 1], z=end_of_ray[:, 2], marker=dict(color='green', size=4))
fig.update_layout(margin=dict(l=20, r=20, t=20, b=20), scene=dict(aspectmode='data'))
fig.show()

## Test canonicalize_camera_poses version

In [ ]:
data['frames'] = canonicalize_camera_poses.canonicalize_camera_poses(data['frames'])
data = canonicalize_camera_poses.center_roi(data)
Ts = np.array([frame['transform_matrix'] for frame in data['frames']])

fig = px.scatter_3d(x=Ts[:, 0, 3], y=Ts[:, 1, 3], z=Ts[:, 2, 3], width=500, height=300)
fig.add_scatter3d(x=[0], y=[0], z=[0], marker=dict(color='green', size=4))
fig.update_layout(margin=dict(l=20, r=20, t=20, b=20), scene=dict(aspectmode='data'))
fig.show()